In [1]:
!pip install opencv-python

# Detecting and tracking moving cars

In [1]:
import cv2
import numpy as np
from IPython.display import display, Image, clear_output

cap = cv2.VideoCapture('Traffic_Laramie_1.mp4')
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=100, detectShadows=False)
roi_y1, roi_y2 = 250, 600  
line_y = roi_y2 - 20  
centroids_memory = []
frame_rate = cap.get(cv2.CAP_PROP_FPS)
car_count = 0
frame_count = 0

def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    boxes = np.array(boxes)
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    pick = []
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,0] + boxes[:,2]
    y2 = boxes[:,1] + boxes[:,3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = idxs[-1]
        pick.append(last)
        suppress = [last]

        for i in idxs[:-1]:
            xx1 = max(x1[last], x1[i])
            yy1 = max(y1[last], y1[i])
            xx2 = min(x2[last], x2[i])
            yy2 = min(y2[last], y2[i])
            w = max(0, xx2 - xx1 + 1)
            h = max(0, yy2 - yy1 + 1)
            overlap = float(w * h) / area[i]

            if overlap > overlapThresh:
                suppress.append(i)

        idxs = np.delete(idxs, np.where(np.isin(idxs, suppress)))
    return boxes[pick].astype("int")


while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    roi = frame[roi_y1:roi_y2, :]
    mask = fgbg.apply(roi)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.dilate(mask, kernel, iterations=2)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 1000:
            x, y, w, h = cv2.boundingRect(cnt)
            aspect_ratio = w / float(h)
            if 1.2 < aspect_ratio < 4.0 and w > 40 and h > 20:
                boxes.append((x, y, w, h))
    merged_boxes = non_max_suppression_fast(boxes, overlapThresh=0.55)
    new_centroids = []
    for (x, y, w, h) in merged_boxes:
        cx = x + w // 2
        cy = y + h // 2 + roi_y1
        new_centroids.append((cx, cy))

        if (
            cy >= roi_y2 - 50 or
            cy <= roi_y1 + 50 or
            cx <= 50 or
            cx >= frame.shape[1] - 50
        ):
            if all(abs(cx - m[0]) > 50 or abs(cy - m[1]) > 30 for m in centroids_memory):
                car_count += 1
                centroids_memory.append((cx, cy))

        cv2.rectangle(frame, (x, y + roi_y1), (x + w, y + h + roi_y1), (0, 255, 0), 2)

    if len(centroids_memory) > 50:
        centroids_memory = centroids_memory[-50:]

    for i in range(0, frame.shape[1], 20):
        if (i // 20) % 2 == 0:
            cv2.line(frame, (i, roi_y1), (i + 10, roi_y1), (0, 0, 255), 2)
            cv2.line(frame, (i, roi_y2), (i + 10, roi_y2), (0, 0, 255), 2)
    cv2.line(frame, (0, roi_y1), (0, roi_y2), (0, 0, 255), 2)
    cv2.line(frame, (frame.shape[1] - 1, roi_y1), (frame.shape[1] - 1, roi_y2), (0, 0, 255), 2)


    cv2.line(frame, (0, roi_y2), (frame.shape[1], roi_y2), (255, 0, 0), 2)  
    cv2.line(frame, (0, roi_y1), (frame.shape[1], roi_y1), (255, 0, 0), 2)  
    cv2.line(frame, (0, roi_y1), (0, roi_y2), (255, 0, 0), 2)               
    cv2.line(frame, (frame.shape[1] - 1, roi_y1), (frame.shape[1] - 1, roi_y2), (255, 0, 0), 2)  

    cv2.putText(frame, f"Cars -> {car_count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(frame, "Focus street", (30, roi_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow('Full Frame with Detection', frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

duration_sec = frame_count / frame_rate
cars_per_minute = car_count / (duration_sec / 60)
print(f"Total cars: {car_count}")
print(f"Cars per minute: {cars_per_minute:.2f}")

Total cars: 16
Cars per minute: 5.40


# Count the number of cars that go from the city downtown to the city centre

In [1]:
import cv2
import numpy as np

video_path = "Traffic_Laramie_2.mp4"
cap = cv2.VideoCapture(video_path)
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=100)

roi_y1, roi_y2 = 300, 460
car_count = 0
min_contour_area = 500
line_position = 400  
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)

detected_centroids = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    roi = frame[roi_y1:roi_y2, :]
    mask = fgbg.apply(roi)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        if cv2.contourArea(cnt) < min_contour_area:
            continue

        x, y, w, h = cv2.boundingRect(cnt)
        cx = x + w // 2
        cv2.rectangle(roi, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.circle(roi, (cx, y + h // 2), 3, (0, 0, 255), -1)
        detected_centroids.append(cx)

        if cx > line_position - 5 and cx < line_position + 5:
            car_count += 1
            
    cv2.line(roi, (line_position, 0), (line_position, roi.shape[0]), (255, 0, 0), 2)
    cv2.putText(roi, f'Cars counted: {car_count}', (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    frame[roi_y1:roi_y2, :] = roi
    cv2.imshow("Task 2 - Press Q to stop", frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

duration_sec = frame_count / frame_rate
cars_per_minute = car_count / (duration_sec / 60)
print(f"Total cars: {car_count}")
print(f"Cars per minute: {cars_per_minute:.2f}")

Total cars: 3
Cars per minute: 1.70
